In [ ]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False


In [ ]:
# importing data from different runs
run_name = 'Run3'

tsg_screen_data = pd.read_excel('./TSGscreen/' + run_name+'.xlsx')
tsg_screen_data.dropna(axis=0)
tsg_screen_data.dtypes
tsg_screen_data['Time'] = pd.to_datetime(tsg_screen_data['Time'],format='%H:%M:%S')
tsg_screen_data.head()
feeder_data = pd.read_excel('./Feederdata/' + run_name+'.xlsx')
feeder_data = feeder_data.dropna(axis=1)
feeder_data['Time'] = pd.to_datetime(feeder_data['Time'],format='%H:%M:%S')
feeder_data.head()
eyecon_data = pd.read_excel('./Eyecondata/' + run_name+'.xlsx')
eyecon_data['Time'] = pd.to_datetime(eyecon_data['Time'],format='%H:%M:%S')
eyecon_data.head()
feed_eyecon = pd.merge_asof(eyecon_data,feeder_data,on='Time', tolerance=pd.Timedelta('2s'))
feed_eyecon = feed_eyecon.dropna(axis=0)
len(feed_eyecon)

combined_Data = pd.merge_asof(feed_eyecon,tsg_screen_data,on='Time', tolerance=pd.Timedelta('2s'))
combined_Data = combined_Data.dropna(axis=0)
combined_Data.columns.values
# plot_cols = [' D_v50', 'Torque','Zone 2','Zone 3','Zone 4','Zone 5','Zone 6','Zone 7','Zone 8']
# plot_cols = ['Zone 2','Zone 3','Zone 4','Zone 5','Zone 6','Zone 7','Zone 8']
plot_cols = [' D_v50', 'Torque']
time_arr = combined_Data.pop(combined_Data.columns.values[1])
plot_features = combined_Data[plot_cols]
plot_features.index = time_arr
_ = plot_features.plot(subplots=True)
combined_Data = combined_Data.drop([' TimeStamp','TimeStamp'],axis=1)
combined_Data = combined_Data.drop([' D_v10', ' D_v90', ' D_n10',' D_n50', ' D_n90', ' Median diameter',' Std deviation',' Shape mean', ' Shape RSD', '7 Massflow','7 Setpoint', '7 ScrewSpeed', '7 AveFeedFactor','7 DriveCommand'],axis=1)


In [ ]:
column_indices = {name: i for i, name in enumerate(combined_Data.columns)}

n = len(combined_Data)
train_combined_Data = combined_Data[0:int(n*0.7)]
val_combined_Data = combined_Data[int(n*0.7):int(n*0.9)]
test_combined_Data = combined_Data[int(n*0.9):]

num_features = combined_Data.shape[1]
train_mean = train_combined_Data.mean()
train_std = train_combined_Data.std()

train_combined_Data = (train_combined_Data - train_mean) / train_std
val_combined_Data = (val_combined_Data - train_mean) / train_std
test_combined_Data = (test_combined_Data - train_mean) / train_std
combined_Data_std = (combined_Data - train_mean) / train_std
combined_Data_std = combined_Data_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=combined_Data_std)
_ = ax.set_xticklabels(combined_Data.keys(), rotation=90)



In [ ]:
train_combined_Data.head()

In [ ]:
y_d50 = np.array(train_combined_Data[' D_v50'])
y_torque = np.array(train_combined_Data['Torque'])
x_MFR = np.array(train_combined_Data['Mass flow rate'])
x_netweight = np.array(train_combined_Data['7 NetWeight'])
x_LFR = np.array(train_combined_Data['Liquid flow rate'])
x_time = np.array(train_combined_Data['Time index'])
x_rpm = np.linspace(train_mean['Actual RPM'],train_mean['Actual RPM'],len(train_combined_Data))
print(x_rpm)

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [ ]:
# reshape into X=t and Y=t+1
# choose a number of time steps
n_steps = 5
x_MFR = x_MFR.reshape((len(x_MFR), 1))
x_time = x_time.reshape((len(x_time), 1))
y_torque = y_torque.reshape((len(y_torque), 1))
# convert into input/output
dataset = np.hstack((x_MFR,x_time,y_torque))
X, y = split_sequences(dataset, n_steps)

In [ ]:
for i in range(len(X)):
	print(X[i], y[i])

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# define model
n_features = 2
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = np.array([[0.5, 0.2], [0.2, -0.1], [1, .5], [.1, .5], [.2, .5]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix,:], sequence[end_ix:out_end_ix,:]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

# choose a number of time steps
n_steps_in, n_steps_out = 50, len(test_combined_Data)

x_MFR = x_MFR.reshape((len(x_MFR), 1))
x_time = x_time.reshape((len(x_time), 1))
x_LFR = x_LFR.reshape((len(x_LFR), 1))
x_rpm = x_rpm.reshape((len(x_rpm), 1))
x_netweight = x_netweight.reshape((len(x_netweight), 1))
y_torque = y_torque.reshape((len(y_torque), 1))
y_d50 = y_d50.reshape((len(y_d50),1))

dataset = np.hstack((x_MFR,x_time,x_LFR,x_rpm,x_netweight,y_torque,y_d50))
X, y = split_sequence(dataset, n_steps_in, n_steps_out)
n_features = 5
X=X[:,:,:5]
y=y[:,:,5:7]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=200, verbose=0)
# print(y.shape)

In [ ]:
test_y_d50 = np.array(test_combined_Data[' D_v50'])
test_y_torque = np.array(test_combined_Data['Torque'])
test_x_MFR = np.array(test_combined_Data['Mass flow rate'])
test_x_netweight = np.array(test_combined_Data['7 NetWeight'])
test_x_LFR = np.array(test_combined_Data['Liquid flow rate'])
test_x_time = np.array(test_combined_Data['Time index'])
test_x_rpm = np.linspace(train_mean['Actual RPM'],train_mean['Actual RPM'],len(test_combined_Data))

test_y_d50 = test_y_d50.reshape(len(test_y_d50),1)
test_y_torque = test_y_torque.reshape(len(test_y_torque),1)
test_x_MFR = test_x_MFR.reshape(len(test_x_MFR),1)
test_x_netweight = test_x_netweight.reshape(len(test_x_netweight),1)
test_x_LFR = test_x_LFR.reshape(len(test_x_LFR),1)
test_x_time = test_x_time.reshape(len(test_x_time),1)
test_x_rpm = test_x_rpm.reshape(len(test_x_rpm),1)



dataset_test = np.hstack((test_x_MFR,test_x_time,test_x_LFR,test_x_rpm,test_x_netweight))
# demonstrate prediction
# x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = dataset_test.reshape((1, len(dataset_test), n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)